# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [1]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

--2016-10-24 14:41:49--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5025226 (4.8M) [text/plain]
Saving to: ‘abq-food-inspections-fy2010.csv.1’

abq-food-inspection 100%[=====================>]   4.79M  --.-KB/s   in 0.1s   

2016-10-24 14:41:50 (38.2 MB/s) - ‘abq-food-inspections-fy2010.csv.1’ saved [5025226/5025226]



## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

<><><

This dataset documents inspections of establishments that sell food or serve food to the public in Albuquerque NM in fiscal year 2010. The dataset contains the 27 variables listed immediately below followed by descriptive statistics for each. The data exhibit imperfections (null values, etc.) that are common in field data but are generally clean and orderly. This data is from the reporting application used on a daily basis by inspectors for the Environmental Health Department, City of Albuquerque NM. 

There are 17,556 records in the dataset. 

Vendors in 2615 locations (FACILITY_NAMEs) with 2835 numbers (FACILITY_KEYs) were inspected at least once. As the most frequent NAME (SMITHS FOOD AND DRUG) is not one of the most frequent KEYs, we see how there could easily be 220 more NAMEs than KEYs.

Fields 3-6 and 10-13 are all elements of the postal address for the site. Field 3 (SITE_ADDRESS) has null values, field 4 (CITY) is ALBUQUERQUE in all records, but field 5 (STATE) in NM in most records, but 87 in at least one record. This indicates that the comma that should separate fields 4 and 5 is missing in at least one record as all zip codes (field 6) begin with 87.

Field 8 OWNER_NAME indicates that the Albuquerque Public Schools own more than twice as many inspected facilities than the next most frequent owner (SMITHS FOOD AND DRUG). The most frequent business type to be inspected by far were restaurants (field 9).

The rest of the dataset is comprised of variable pairs (codes and descriptions) that report on the inspections and their outcomes: Program Category, Inspection Type, Action, Result and Violation. Counts of records for each of these pairs are presented in tables and discussed after the descriptive statistics below.

In [2]:
# Here are the variables (columns)

!csvcut -n abq-food-inspections-fy2010.csv

  1: FACILITY_NAME
  2: FACILITY_KEY
  3: SITE_ADDRESS
  4: CITY
  5: STATE
  6: ZIP
  7: OWNER_KEY
  8: OWNER_NAME
  9: NATURE_OF_BUSINESS
 10: STREET_NUMBER
 11: STREET_NAME
 12: STREET_TYPE
 13: POST_DIRECTIONAL
 14: PHONE
 15: PROGRAM_CATEGORY
 16: PROGRAM_CATEGORY_DESCRIPTION
 17: INSPECTION_DATE
 18: INSPECTION_TYPE
 19: INSPECTION_DESC
 20: SERIAL_NUM
 21: ACTION_CODE
 22: ACTION_DESC
 23: RESULT_CODE
 24: RESULT_DESC
 25: VIOLATION_CODE
 26: VIOLATION_DESC
 27: INSPECTION_MEMO


In [3]:
# and descriptive statistics for each variable.

!csvstat abq-food-inspections-fy2010.csv

  1. FACILITY_NAME
	<class 'str'>
	Nulls: False
	Unique values: 2615
	5 most frequent values:
		SMITHS FOOD AND DRUG CENTERS INC:	154
		ISOTOPES PARK - OVATIONS:	93
		PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR:	85
		ISLETA AMPHITHEATER:	68
		ECHO INC:	68
	Max length: 77
  2. FACILITY_KEY
	<class 'int'>
	Nulls: False
	Min: 13
	Max: 101482
	Sum: 820405773
	Mean: 46730.79135338346
	Median: 50866.0
	Standard Deviation: 32659.794578837962
	Unique values: 2835
	5 most frequent values:
		46323:	93
		65863:	85
		80949:	68
		32817:	68
		91825:	66
  3. SITE_ADDRESS
	<class 'str'>
	Nulls: True
	Unique values: 2456
	5 most frequent values:
		1601 AVENIDA CESAR CHAVEZ SE:	94
		644 OLD COORS BLVD SW:	85
		8601 CENTRAL AV NE:	77
		2200 SUNPORT BLVD SE:	75
		5210 CENTRAL AV SE:	73
	Max length: 35
  4. CITY
	<class 'str'>
	Nulls: False
	Values: ALBUQUERQUE
  5. STATE
	<class 'str'>
	Nulls: False
	Values: 87, NM
  6. ZIP
	<class 'str'>
	Nulls: False
	Unique values: 48
	5 most frequent values:
		87110

<><><

The Program Category listing below shows that category 0411 is missing a description.

In [4]:
!csvcut -c15,16 abq-food-inspections-fy2010.csv | csvsort | uniq -c | csvlook

|---------------------------+-------------------------------------------|
|        1 PROGRAM_CATEGORY | PROGRAM_CATEGORY_DESCRIPTION              |
|---------------------------+-------------------------------------------|
|      748 0402             | Childcare                                 |
|      343 0403             | Bakery                                    |
|      357 0404             | Institutional Kitchen                     |
|      489 0405             | Food Processor                            |
|     7969 0406             | Food Service Establishment                |
|      288 0407             | Limited Menu                              |
|      832 0408             | School                                    |
|      319 0409             | Non School                                |
|       73 0410             | NOT FOR PROFIT KITCHEN PERMIT             |
|       96 0411             |                                           |
|      832 0501             | Bar     

<><><

INSPECTION_TYPE is null for all, so INSPECTION_DESC actually conveys the type.

In [5]:
!csvcut -c18,19 abq-food-inspections-fy2010.csv | csvsort | uniq -c | csvlook

|--------------------------+-----------------------------------|
|        1 INSPECTION_TYPE | INSPECTION_DESC                   |
|--------------------------+-----------------------------------|
|      439                 | ADDITIONAL SERVICE PROVIDED       |
|        1                 | COMPLAINT FOOD INSPECTION         |
|        1                 | E-MAIL                            |
|     1987                 | EMERGENCY RESPONSE                |
|        5                 | FIRE CALL                         |
|       75                 | FOLLOW-UP DOCUMENTATION RECEIVED  |
|      143                 | FOOD FOLLOW-UP INSPECTION         |
|       36                 | IMMINENT HEALTH HAZARD            |
|       54                 | MANAGERIAL ASSISTANCE             |
|      377                 | MOBILE UNIT FOOD INSPECTION       |
|     1005                 | New Business Inspection           |
|      247                 | PHONE CALL                        |
|        4               

<><><

ACTION_CODE 00 means no action was taken. Else, a negative action taken in ~1.6% of cases appears to show that both the food distribution system and the inspections regime are both relatively healthy.

In [6]:
!csvcut -c21,22 abq-food-inspections-fy2010.csv | csvsort | uniq -c | csvlook

|----------------------+-------------------------------------|
|        1 ACTION_CODE | ACTION_DESC                         |
|----------------------+-------------------------------------|
|     2800 00          | NON-GRADED                          |
|        1 03          | FACILITY CLOSED AT TIME OF SERVICE  |
|      264 06          | UNSATISFACTORY                      |
|      382 09          | ANNUAL STICKER ISSUED               |
|       21 10          | ANNUAL STICKER NOT ISSUED           |
|    14080 11          | APPROVED                            |
|        3 12          | NO PERMIT APPLICATION ISSUED        |
|        5 17          | FOOD PERMIT SUSPENSION              |
|----------------------+-------------------------------------|


<><><

RESULT_CODEs appear to be generally consistent with Actions.

In [7]:
!csvcut -c23,24 abq-food-inspections-fy2010.csv | csvsort | uniq -c | csvlook

|----------------------+----------------------------------------|
|        1 RESULT_CODE | RESULT_DESC                            |
|----------------------+----------------------------------------|
|     2416 00          | NOT APPLICABLE                         |
|       24 02          | NOT IN COMPLIANCE                      |
|    14389 03          | IN COMPLIANCE                          |
|        3 04          | NOTICE                                 |
|      214 05          | DOWNGRADE                              |
|       69 06          | UPGRADE                                |
|        2 07          | PERMIT SUSPENSION                      |
|        1 08          | VOLUNTARY CLOSURE                      |
|        1 10          | PERMISSION TO OPERATE                  |
|       19 11          | FACILITY CLOSED AT TIME OF INSPECTION  |
|       11 12          | PERMIT REINSTATEMENT                   |
|       90 13          | PHONE CALL                             |
|       45

<><><

VIOLATION_CODEs and DESCriptions are null for 1008 observations. It may be evident from inspection ACTIONs or RESULTs. It is strange that there are exactly 5 observations each of CODE 04  00 - 04  73. As with the null observations, the meaning of this pattern is likely evident from other variables for those observations.

In [8]:
!csvcut -c25,26 abq-food-inspections-fy2010.csv | csvsort | uniq -c | csvlook

|-------------------------+-----------------------------------------------------|
|        1 VIOLATION_CODE | VIOLATION_DESC                                      |
|-------------------------+-----------------------------------------------------|
|     1008                |                                                     |
|        1                | Additional Comments                                 |
|     3429 00             | No Violations Found                                 |
|      196 01             | Cold Holding                                        |
|      170 02             | Hot Holding                                         |
|        1 02 15A0204     | OTHER - MISCELLEANOUS                               |
|       40 03             | Reheating Temp                                      |
|        1 04             | Cooking Temperatures                                |
|        5 04  00         | No Violations Found                                 |
|        5 04  0

<><><

<><><

<><><


## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

<><><

We shall use postgreSQL on datanotebook.org to directly define our schema. First we need to restart postgreSQL ...

In [9]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


<><><

Next we create database ex04 as the default user ...

In [10]:
!createdb -U dbuser ex04

createdb: database creation failed: ERROR:  database "ex04" already exists


<><><

We then activate the `ipython-sql` extension ...

In [11]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


<><><

and connect to ex04 specifying the postgreSQL backend, localhost and port ...

In [12]:
%sql postgresql://dbuser@localhost:5432/ex04

'Connected: dbuser@ex04'

Now that we're connected to the ex04 database, we can specify the structure of the data table that will hold the data. I'm calling the table ex04 -- probably not a very good idea -- but SQL takes it all in stride.

We'll keep all variables initially. Later, we'll drop CITY (all Albuquerque), STATE (all should be NM, we know it's not, but we're not going to clean the data), INSPECTION_TYPE (all missing) and INSPECTION_MEMO (all missing). I'm sure I could drop these variables right here. But since I'm new to SQL I'll be cautious and get it all in to begin.

In [13]:
%%sql
DROP TABLE IF EXISTS ex04;
CREATE TABLE ex04 (
    fac VARCHAR(77),
    fac_key CHAR(6),
    site_addr VARCHAR(35),
    city CHAR(11),
    state CHAR(2),
    zipc CHAR(10),
    owner_key CHAR(5),
    owner VARCHAR(82),
    bus VARCHAR(25),
    st_num CHAR(7),
    st VARCHAR(24),
    st_typ CHAR(4),
    post CHAR(2),
    phone CHAR(15),
    pgm_num CHAR(4),
    pgm_desc VARCHAR(40),
    insp_date TIMESTAMP,
    insp_type CHAR(1),
    insp_desc VARCHAR(32),
    serial CHAR(9),
    action_code CHAR(2),
    action_desc VARCHAR(34),
    result_code CHAR(2),
    result_desc VARCHAR(37), 
    violation_code VARCHAR(10),
    violation_desc VARCHAR(50),
    insp_memo CHAR(1)
)

Done.
Done.


[]

<><><

Apparently, the COPY (table) FROM statement below requires an absolute path, so we provide it ... 

In [14]:
!pwd

/home/jovyan/work


In [15]:
%%sql
COPY ex04 FROM '/home/jovyan/work/abq-food-inspections-fy2010.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

17556 rows affected.


[]

<><><

Well that looks perfect to me. Let's count all rows explicitly ...

In [16]:
%%sql
SELECT COUNT(*) FROM ex04;

1 rows affected.


count
17556


<><><

Now let's see if we can extract the schema we just put into table ex04 ...

In [17]:
%%sql
SELECT column_name, data_type, character_maximum_length, is_nullable
FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'ex04';

27 rows affected.


column_name,data_type,character_maximum_length,is_nullable
fac,character varying,77,YES
fac_key,character,6,YES
site_addr,character varying,35,YES
city,character,11,YES
state,character,2,YES
zipc,character,10,YES
owner_key,character,5,YES
owner,character varying,82,YES
bus,character varying,25,YES
st_num,character,7,YES


<><><

Now THAT is absolutely WILD!

<><><

<><><

<><><


## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

<><><

I'm basically interested in one thing ...if any neighborhood has a substantially higher rate of bad citations per inspection. I'll use the number of records in the database as a measure of inspections. This is not entirely accurate, but I'm just taking a quick & dirty first look. I'll use zip code to proxy for neighborhood and start by displaying counts of all records in each zip code. I'll then manually compute a new neighborhood indicator from contiguous zip codes containing 1,000-2000 records each. 


In [18]:
%%sql
SELECT zipc, COUNT (*) FROM ex04
GROUP BY zipc
ORDER by zipc
LIMIT 100;

48 rows affected.


zipc,count
87017,3
87100,3
87101,15
87102,1601
871023114,9
87103,35
87104,527
87105,533
87106,1568
87107,1186


Create a new variable for neighborhood (hood)

In [19]:
%%sql
ALTER TABLE ex04
ADD COLUMN hood INTEGER;

Done.


[]

Recode zipc into the 12 natural contiguous groups of zip codes of 1000-2000 records...

In [20]:
%%sql
UPDATE ex04
SET hood = 1
WHERE zipc < 87103;

(psycopg2.ProgrammingError) operator does not exist: character < integer
LINE 3: WHERE zipc < 87103;
                   ^
HINT:  No operator matches the given name and argument type(s). You might need to add explicit type casts.
 [SQL: 'UPDATE ex04\nSET hood = 1\nWHERE zipc < 87103;']


Well, that was my plan anyway. It appears that SQL doesn't have a straightforward recode function -- not one that I could find anyway. So I'm attempting to use brute force and not having any luck either. Even though SQL sorts zip codes that are strings it does not compute on them. I'm hitting my head against a concrete wall trying to do a really simple thing: a recode or a conditional assignment (If x then y). If I can't get observations into comparable groups then any computed rate of bad citations will not be stable enough across groups for comparison. 

So, I'm stuck. I'm sure there are several really easy fixes. But I can't find even one. There isn't even a reference to recoding a variable in `Learning SQL by Alan Beaulieu` and only scant reference to conditional logic. So it's essentally as if I've gone down a rabbit hole. I probably need to go back and change the variable type of zipc to integer. But to retain my sanity right now I'm going to abandon. 

I'm going to treat this frustration as though it's an essential part of the learning process. Strangely, that may actually be true.

## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table